1. Написати функцію пошуку випадкового простого числа з заданого
інтервалу або заданої довжини, використовуючи датчик випадкових
чисел та тести перевірки на простоту. В якості датчика випадкових
чисел використовуйте вбудований генератор псевдовипадкових чисел
вашої мови програмування. В якості тесту перевірки на простоту
рекомендовано використовувати тест Міллера-Рабіна із попередніми
пробними діленнями. Тести необхідно реалізовувати власноруч,
використання готових реалізацій тестів не дозволяється

In [20]:
import random

# Function to check if a number is probably prime using Miller-Rabin test with preliminary divisions
def is_prime(n, k=10):
    # Handle small numbers directly
    if n in (2, 3):
        return True
    if n <= 1 or n % 2 == 0:
        return False

    # Preliminary divisions by first few primes
    for p in [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]:
        if n % p == 0 and n != p:
            return False

    # Write n as 2^r * d + 1 with d odd
    r, d = 0, n - 1
    while d % 2 == 0:
        r += 1
        d //= 2

    # Witness loop
    for _ in range(k):
        a = random.randint(2, n - 2)
        x = pow(a, d, n)
        if x == 1 or x == n - 1:
            continue

        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False

    return True

# Function to generate a random prime number within a given range or of a given bit length
def generate_random_prime(min_val=None, max_val=None, bit_length=None):
    if bit_length is not None:
        min_val = 2 ** (bit_length - 1)
        max_val = 2 ** bit_length - 1

    # Ensure min_val and max_val are set
    if min_val is None or max_val is None:
        raise ValueError("Either range or bit length must be specified")

    # Generate prime within the range
    while True:
        candidate = random.randint(min_val, max_val)
        if is_prime(candidate):
            return candidate

# Example usage
generate_random_prime(bit_length=16)  # Generate a random prime number with a bit length of 16

45691

2. За допомогою цієї функції згенерувати дві пари простих чисел p, q і
p1 , q1 довжини щонайменше 256 біт. При цьому пари чисел беруться
так, щоб pq <= p1q1 ; p і q – прості числа для побудови ключів
абонента А, 1 p і q1 – абонента B

In [21]:
# Generate two pairs of prime numbers (p, q) and (p1, q1) of at least 256 bits
p, q = generate_random_prime(bit_length=256), generate_random_prime(bit_length=256)
p1, q1 = generate_random_prime(bit_length=256), generate_random_prime(bit_length=256)

# Ensure pq <= p1q1
while p * q > p1 * q1:
    p, q = generate_random_prime(bit_length=256), generate_random_prime(bit_length=256)
    p1, q1 = generate_random_prime(bit_length=256), generate_random_prime(bit_length=256)

p, q, p1, q1, p * q, p1 * q1


(67371973346209419211789180669081363225326293303100421965387021702825761817179,
 59678562125646611280302744184045063338864632494171807540279605231651169688397,
 87144972348380098875217851672633659519270968515987624014459808885927429563641,
 109500995730372528928337379180340992144271301924626217818047591078035411808521,
 4020662496869166435632843979765416721302177787094346899681706220934698943716427818815088701768111017070855349926288997853682345457040241720225589911572063,
 9542461245043401302511140358937085523994410764980237612926904866361322098050268510754200799467718971918914602878645972754337016645559461262207369675584961)

3. Написати функцію генерації ключових пар для RSA. Після
генерування функція повинна повертати та/або зберігати секретний
ключ (d, p,q) та відкритий ключ (n,e) . За допомогою цієї функції
побудувати схеми RSA для абонентів А і B – тобто, створити та
зберегти для подальшого використання відкриті ключі (e,n) , (e1, n1)
та секретні d і d1

In [25]:
def generate_rsa_key_pair(p, q, e=65537):
    n = p * q
    phi = (p - 1) * (q - 1)

    # Ensure that e and phi are coprime
    if gcd(e, phi) != 1:
        raise ValueError("e and phi are not coprime.")

    # Calculate d
    d = pow(e, -1, phi)

    # The public key is (e, n) and the secret key is (d, p, q)
    return (e, n), (d, p, q)

# Generate RSA key pairs for subscribers A and B
public_key_A, secret_key_A = generate_rsa_key_pair(p, q)
public_key_B, secret_key_B = generate_rsa_key_pair(p1, q1)

(public_key_A, secret_key_A, public_key_B, secret_key_B)


((65537,
  4020662496869166435632843979765416721302177787094346899681706220934698943716427818815088701768111017070855349926288997853682345457040241720225589911572063),
 (1721160357502852806074117488629610237211538486914748955102770465970718508109324779194724363446128125257047047931470458799789671931862650170778066964550793,
  67371973346209419211789180669081363225326293303100421965387021702825761817179,
  59678562125646611280302744184045063338864632494171807540279605231651169688397),
 (65537,
  9542461245043401302511140358937085523994410764980237612926904866361322098050268510754200799467718971918914602878645972754337016645559461262207369675584961),
 (754957681241894437547038508950498015501335426040595877489448734792307476362812191875063594806978675183905267899917663336115520218156526172631216937537473,
  87144972348380098875217851672633659519270968515987624014459808885927429563641,
  109500995730372528928337379180340992144271301924626217818047591078035411808521))

4. Написати програму шифрування, розшифрування і створення
повідомлення з цифровим підписом для абонентів А і B. Кожна з
операцій (шифрування, розшифрування, створення цифрового
підпису, перевірка цифрового підпису) повинна бути реалізована
окремою процедурою, на вхід до якої повинні подаватись лише ті
ключові дані, які необхідні для її виконання
За допомогою датчика випадкових чисел вибрати відкрите
повідомлення M і знайти криптограму для абонентів А и B,
перевірити правильність розшифрування. Скласти для А і B
повідомлення з цифровим підписом і перевірити його

In [27]:
# Function to encrypt a message using RSA
def rsa_encrypt(message, public_key):
    e, n = public_key
    encrypted_message = [pow(ord(char), e, n) for char in message]
    return encrypted_message

# Function to decrypt a message using RSA
def rsa_decrypt(encrypted_message, secret_key):
    d, p, q = secret_key
    n = p * q  # Calculate n from p and q

    decrypted_message = ''.join([chr(pow(char, d, n)) for char in encrypted_message])
    return decrypted_message

# Function to create a digital signature using RSA
def rsa_sign(message, secret_key):
    d, p, q = secret_key
    hash_of_message = simple_hash(message)
    signature = pow(hash_of_message, d, p * q)
    return signature

# Function to verify a digital signature using RSA
def rsa_verify_signature(message, signature, public_key):
    e, n = public_key
    hash_of_message = simple_hash(message)
    hash_from_signature = pow(signature, e, n)
    return hash_of_message == hash_from_signature

# Generating a random message
random_message = "Hello, this is a secure message"

# Encrypt the message for both A and B
encrypted_message_A = rsa_encrypt(random_message, public_key_A)
encrypted_message_B = rsa_encrypt(random_message, public_key_B)

# Decrypt the message for both A and B
decrypted_message_A = rsa_decrypt(encrypted_message_A, secret_key_A)
decrypted_message_B = rsa_decrypt(encrypted_message_B, secret_key_B)

# Create a digital signature for both A and B
signature_A = rsa_sign(random_message, secret_key_A)
signature_B = rsa_sign(random_message, secret_key_B)

# Verify the digital signature for both A and B
verification_A = rsa_verify_signature(random_message, signature_A, public_key_A)
verification_B = rsa_verify_signature(random_message, signature_B, public_key_B)

(decrypted_message_A, decrypted_message_B, verification_A, verification_B)


('Hello, this is a secure message',
 'Hello, this is a secure message',
 True,
 True)

5. За допомогою раніше написаних на попередніх етапах програм
організувати роботу протоколу конфіденційного розсилання ключів з
підтвердженням справжності по відкритому каналу за допомогою
алгоритму RSA. Протоколи роботи кожного учасника (відправника та
приймаючого) повинні бути реалізовані у вигляді окремих процедур,
на вхід до яких повинні подаватись лише ті ключові дані, які необхідні
для виконання. Перевірити роботу програм для випадково обраного
ключа 0 < k < n

In [28]:
# Function for the sender to encrypt and sign a key
def sender_encrypt_and_sign(key, sender_secret_key, receiver_public_key):
    # Encrypt the key using the receiver's public key
    encrypted_key = rsa_encrypt(str(key), receiver_public_key)

    # Sign the encrypted key using the sender's secret key
    signature = rsa_sign(str(key), sender_secret_key)

    return encrypted_key, signature

# Function for the receiver to decrypt and verify the key
def receiver_decrypt_and_verify(encrypted_key, signature, sender_public_key, receiver_secret_key):
    # Decrypt the key using the receiver's secret key
    decrypted_key = rsa_decrypt(encrypted_key, receiver_secret_key)

    # Verify the signature using the sender's public key
    is_valid_signature = rsa_verify_signature(decrypted_key, signature, sender_public_key)

    return decrypted_key, is_valid_signature

# Simulate the process of sending a key from A to B
# Generate a random key 0 < k < n
k = random.randint(1, public_key_B[1] - 1)

# Sender (A) encrypts and signs the key
encrypted_key, signature = sender_encrypt_and_sign(k, secret_key_A, public_key_B)

# Receiver (B) decrypts the key and verifies the signature
decrypted_key, is_valid_signature = receiver_decrypt_and_verify(encrypted_key, signature, public_key_A, secret_key_B)

(k, decrypted_key, is_valid_signature)


(4640261432800888498121490316986318912133771596966748542050403015242232299777976827520941950369558862521829833601976425405344390702671997096966116901741082,
 '4640261432800888498121490316986318912133771596966748542050403015242232299777976827520941950369558862521829833601976425405344390702671997096966116901741082',
 True)